In [5]:
! pip install pymysql

In [6]:
import pymysql
import json
import requests
import jellyfish
from datetime import datetime
import re

In [7]:
import pandas
df = pandas.read_csv("../7103.csv")

In [14]:
df

,Country,State,State LGD code,Year,Normal tax payers_avg,Normal tax payers_max,Normal tax payers_min,Normal tax payers_sum,Normal tax payers_count,Normal tax payers_stddev,...,Unique identification number (uin) holders_max,Unique identification number (uin) holders_min,Unique identification number (uin) holders_sum,Unique identification number (uin) holders_count,Unique identification number (uin) holders_stddev,Unique identification number (uin) holders_LandAreaWeight,Unique identification number (uin) holders_TotalPopulationWeight,Unique identification number (uin) holders_NumberOfHouseholdsWeight,Unique identification number (uin) holders_TotalPopulationMaleWeight,Unique identification number (uin) holders_TotalPopulationFemaleWeight
0,India,Jammu And Kashmir,1,"Financial Year (Apr - Mar), 2022",110462.0,110462.0,110462.0,110462.0,1,NaN,...,10.0,10.0,10.0,1,NaN,10.0,10.0,10.0,10.0,10.0
1,India,Himachal Pradesh,2,"Financial Year (Apr - Mar), 2022",99587.0,99587.0,99587.0,99587.0,1,NaN,...,25.0,25.0,25.0,1,NaN,25.0,25.0,25.0,25.0,25.0
2,India,Punjab,3,"Financial Year (Apr - Mar), 2022",344609.0,344609.0,344609.0,344609.0,1,NaN,...,15.0,15.0,15.0,1,NaN,15.0,15.0,15.0,15.0,15.0
3,India,Chandigarh,4,"Financial Year (Apr - Mar), 2022",27759.0,27759.0,27759.0,27759.0,1,NaN,...,3.0,3.0,3.0,1,NaN,3.0,3.0,3.0,3.0,3.0
4,India,Uttarakhand,5,"Financial Year (Apr - Mar), 2022",157239.0,157239.0,157239.0,157239.0,1,NaN,...,11.0,11.0,11.0,1,NaN,11.0,11.0,11.0,11.0,11.0
5,India,Haryana,6,"Financial Year (Apr - Mar), 2022",482254.0,482254.0,482254.0,482254.0,1,NaN,...,31.0,31.0,31.0,1,NaN,31.0,31.0,31.0,31.0,31.0
6,India,Delhi,7,"Financial Year (Apr - Mar), 2022",758918.0,758918.0,758918.0,758918.0,1,NaN,...,316.0,316.0,316.0,1,NaN,316.0,316.0,316.0,316.0,316.0
7,India,Rajasthan,8,"Financial Year (Apr - Mar), 2022",672438.0,672438.0,672438.0,672438.0,1,NaN,...,86.0,86.0,86.0,1,NaN,86.0,86.0,86.0,86.0,86.0
8,India,Uttar Pradesh,9,"Financial Year (Apr - Mar), 2022",1377315.0,1377315.0,1377315.0,1377315.0,1,NaN,...,234.0,234.0,234.0,1,NaN,234.0,234.0,234.0,234.0,234.0
9,India,Bihar,10,"Financial Year (Apr - Mar), 2022",484279.0,484279.0,484279.0,484279.0,1,NaN,...,129.0,129.0,129.0,1,NaN,129.0,129.0,129.0,129.0,129.0


In [8]:
timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="iia_group2",
  host="world-athletics-world-athletics.h.aivencloud.com",
  password="AVNS_q4qfMZEYhnIhreAFwYi",
  read_timeout=timeout,
  port=24936,
  user="avnadmin",
  write_timeout=timeout,
)


In [9]:
states = []
cursor = connection.cursor()
cursor.execute("select state from States")
x = cursor.fetchall()
cursor.close()

In [10]:
unique_state_list = [i["state"] for i in x ]

In [106]:
# x

In [11]:
def check_similarity(col_name_sql,col_name_api):
    pass

def match_state_name(input_state, unique_states_list):
    # Initialize variables to store the best match and highest score
    best_match = None
    highest_score = 0

    # Iterate through the unique states list and calculate Jaro-Winkler scores
    for state in unique_states_list:
        similarity_score = jellyfish.jaro_winkler_similarity(input_state, state)
        if similarity_score > highest_score:
            highest_score = similarity_score
            best_match = state
        # print(similarity_score, state)

    # Return the best matching state name
    if(highest_score < 0.7):
        return None
    return best_match

def get_start_date(string:str):
    try:
        temp = re.split(r"[(),-]",string)
        date_string = "1 " + temp[1] + " " + temp[-1] 
        date_object = datetime.strptime(date_string, "%d %b %Y").date()
        return date_object
    except:
        return None

def get_end_date(string:str):
    try:
        temp = re.split(r"[(),-]",string)
        date_string = "1 " + temp[2] + " " + str(int(temp[-1]) +1)
        date_object = datetime.strptime(date_string, "%d %b %Y").date()
    except:
        return None
    return date_object


In [12]:
def get_month(string: str):
    try:
        temp = re.split(r"[(),-]",string)
        
        date_string = "1 " + temp[-2] + " " + str(int(temp[-1]))
        # print(date_string)
        date_object = datetime.strptime(date_string, "%d %B %Y").date()
    except:
        return None
    return date_object


In [197]:
get_month("December, 2019")

datetime.date(2019, 12, 1)

In [177]:
get_end_date("Financial Year (Apr - Mar), 2017")

datetime.date(2018, 3, 1)

In [15]:
df_per_year = df.drop(["State LGD code","Country"],axis=1)

In [17]:
# df_per_year

In [166]:
# df_per_year
df_per_month = df.drop(["Year","Gst ( goods and service tax ) return type","State lgd code","Country"],axis=1)

In [18]:
# df_per_month

In [ ]:
# import numpy as np
df_per_year = df_per_year.groupby(["State","Year"]).mean().reset_index()

In [20]:
for i in df_per_year.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] != "avg"):
        print(df_per_year[i])
        df_per_year = df_per_year.drop([i],axis=1)

In [170]:
for i in df_per_month.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] != "avg"):
        print(df_per_month[i])
        df_per_month = df_per_month.drop([i],axis=1)

In [21]:
df_per_year

,State,Year,Normal tax payers_avg,Composition tax payers_avg,Input service distributor_avg,Casual tax payers_avg,Tax collected at source_avg,Tax deducted at source_avg,Non resident taxpayers_avg,Online information and database access or retrieval services from a place outside india to a non-taxable online recipient_avg,Unique identification number (uin) holders_avg
0,Jammu And Kashmir,"Financial Year (Apr - Mar), 2022",110462.0,8499.0,10.0,5.0,151.0,2593.0,0.0,0.0,10.0
1,Himachal Pradesh,"Financial Year (Apr - Mar), 2022",99587.0,17679.0,33.0,9.0,207.0,1614.0,0.0,0.0,25.0
2,Punjab,"Financial Year (Apr - Mar), 2022",344609.0,36606.0,83.0,27.0,446.0,3369.0,1.0,0.0,15.0
3,Chandigarh,"Financial Year (Apr - Mar), 2022",27759.0,1609.0,84.0,9.0,214.0,760.0,0.0,0.0,3.0
4,Uttarakhand,"Financial Year (Apr - Mar), 2022",157239.0,34265.0,47.0,6.0,352.0,3480.0,0.0,0.0,11.0
5,Haryana,"Financial Year (Apr - Mar), 2022",482254.0,15470.0,608.0,20.0,941.0,2547.0,4.0,0.0,31.0
6,Delhi,"Financial Year (Apr - Mar), 2022",758918.0,16325.0,1045.0,83.0,1381.0,2842.0,4.0,0.0,316.0
7,Rajasthan,"Financial Year (Apr - Mar), 2022",672438.0,134913.0,115.0,8.0,686.0,19619.0,1.0,0.0,86.0
8,Uttar Pradesh,"Financial Year (Apr - Mar), 2022",1377315.0,338879.0,444.0,26.0,1300.0,23645.0,1.0,0.0,234.0
9,Bihar,"Financial Year (Apr - Mar), 2022",484279.0,108500.0,107.0,8.0,374.0,7066.0,0.0,0.0,129.0


In [23]:
# df_per_year

In [24]:
df_per_year["state"] = df_per_year.apply(lambda x : match_state_name(x["State"],unique_state_list),axis=1)
df_per_year["start_date"] = df_per_year.apply(lambda x : get_start_date(x["Year"]),axis=1)
df_per_year["end_date"] = df_per_year.apply(lambda x : get_end_date(x["Year"]),axis=1)


In [35]:
col_name = ["state","start_date","end_date"]
for i in df_per_year.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] == "avg"):
        print(df_per_year[i])
        df_per_year[x[0]] = df_per_year[i]
        col_name.append(x[0])

0      110462.0
1       99587.0
2      344609.0
3       27759.0
4      157239.0
5      482254.0
6      758918.0
7      672438.0
8     1377315.0
9      484279.0
10       9384.0
11      14641.0
12       7740.0
13      11629.0
14       7562.0
15      26898.0
16      28175.0
17     181474.0
18     637786.0
19     170659.0
20     278441.0
21     133819.0
22     421752.0
23    1012153.0
24    1499972.0
25     303295.0
26     857399.0
27      38239.0
28        228.0
29     341278.0
30    1007706.0
31      20581.0
32       4243.0
33     423698.0
34       6023.0
35      14743.0
Name: Normal tax payers_avg, dtype: float64
0       8499.0
1      17679.0
2      36606.0
3       1609.0
4      34265.0
5      15470.0
6      16325.0
7     134913.0
8     338879.0
9     108500.0
10       792.0
11      2258.0
12      2065.0
13      1502.0
14       167.0
15      2916.0
16      2246.0
17     36586.0
18     58056.0
19     17589.0
20     28036.0
21     34105.0
22     52846.0
23     96371.0
24    122786.0
25   

In [36]:
df_per_year[col_name]

,state,start_date,end_date,Normal tax payers,Composition tax payers,Input service distributor,Casual tax payers,Tax collected at source,Tax deducted at source,Non resident taxpayers,Online information and database access or retrieval services from a place outside india to a non-taxable online recipient,Unique identification number (uin) holders
0,jammu and kashmir,2022-04-01,2023-03-01,110462.0,8499.0,10.0,5.0,151.0,2593.0,0.0,0.0,10.0
1,himachal pradesh,2022-04-01,2023-03-01,99587.0,17679.0,33.0,9.0,207.0,1614.0,0.0,0.0,25.0
2,punjab,2022-04-01,2023-03-01,344609.0,36606.0,83.0,27.0,446.0,3369.0,1.0,0.0,15.0
3,chandigarh,2022-04-01,2023-03-01,27759.0,1609.0,84.0,9.0,214.0,760.0,0.0,0.0,3.0
4,uttarakhand,2022-04-01,2023-03-01,157239.0,34265.0,47.0,6.0,352.0,3480.0,0.0,0.0,11.0
5,haryana,2022-04-01,2023-03-01,482254.0,15470.0,608.0,20.0,941.0,2547.0,4.0,0.0,31.0
6,delhi,2022-04-01,2023-03-01,758918.0,16325.0,1045.0,83.0,1381.0,2842.0,4.0,0.0,316.0
7,rajasthan,2022-04-01,2023-03-01,672438.0,134913.0,115.0,8.0,686.0,19619.0,1.0,0.0,86.0
8,uttar pradesh,2022-04-01,2023-03-01,1377315.0,338879.0,444.0,26.0,1300.0,23645.0,1.0,0.0,234.0
9,bihar,2022-04-01,2023-03-01,484279.0,108500.0,107.0,8.0,374.0,7066.0,0.0,0.0,129.0


In [38]:
df_per_year[col_name].to_csv("trans_GST_info_per_year_data.csv")

In [ ]:
df_per_month["state"] = df_per_month.apply(lambda x : match_state_name(x["State"],unique_state_list),axis=1)
df_per_month["date"] = df_per_month.apply(lambda x : get_month(x["Month"]),axis=1)
df_per_month["registered_payer"] = df_per_month.apply(lambda x : x["Gst ( goods and service tax ) payers registered before due date_avg"] +x["Gst ( goods and service tax ) payers registered after due date_avg"] ,axis=1) 
df_per_month["eligible_payer"] = df_per_month.apply(lambda x : x["Payer eligible for gst ( goods and service tax ) registration_avg"],axis=1)


In [157]:
df_per_year_transformed = df_per_year[["state","start_date","end_date","registered_payer","eligible_payer"]]

In [205]:
df_per_month_transformed =  df_per_month[["state","date","registered_payer","eligible_payer"]]

In [208]:
df_per_month_transformed.dropna()

,state,date,registered_payer,eligible_payer
0,chandigarh,2021-02-01,12708.0,12710.0
1,sikkim,2019-12-01,6056.0,7450.0
2,sikkim,2019-12-01,6486.0,7450.0
3,sikkim,2020-01-01,2761.0,3908.0
4,jammu and kashmir,2017-07-01,45971.0,52455.0
...,...,...,...,...
4243,the dadra and nagar haveli and daman and diu,2022-03-01,13594.0,14581.0
4244,the dadra and nagar haveli and daman and diu,2022-04-01,10632.0,11569.0
4245,the dadra and nagar haveli and daman and diu,2022-04-01,10572.0,11569.0
4246,the dadra and nagar haveli and daman and diu,2022-05-01,10308.0,11640.0


In [164]:
df_per_year_transformed.dropna()

,state,start_date,end_date,registered_payer,eligible_payer
0,andaman and nicobar islands,2017-04-01,2018-03-01,2596.500000,3388.833333
1,andaman and nicobar islands,2018-04-01,2019-03-01,2871.958333,4107.000000
2,andaman and nicobar islands,2019-04-01,2020-03-01,2914.541667,3570.041667
3,andaman and nicobar islands,2020-04-01,2021-03-01,3049.166667,3627.708333
4,andaman and nicobar islands,2021-04-01,2022-03-01,3207.208333,3562.750000
...,...,...,...,...,...
211,west bengal,2018-04-01,2019-03-01,375588.041667,437264.333333
212,west bengal,2019-04-01,2020-03-01,411276.708333,492672.916667
213,west bengal,2020-04-01,2021-03-01,408896.416667,511896.625000
214,west bengal,2021-04-01,2022-03-01,404378.250000,455467.500000


In [209]:
df_per_year_transformed.to_csv("trans_register_payers_per_year_data.csv")
df_per_month_transformed.to_csv("trans_register_payers_per_month_data.csv")